# Training with ssa preprocessed straming data on online-boosting model  

In [ ]:
from pyts.decomposition import SingularSpectrumAnalysis
from sklearn.preprocessing import StandardScaler
import pandas as pd 
import numpy as np 
import datetime
import matplotlib.pyplot as plt
import glob
import itertools
from skmultiflow.data.data_stream import DataStream
from skmultiflow.meta import AdaptiveRandomForestClassifier, StreamingRandomPatchesClassifier,\
OnlineBoostingClassifier, OnlineSMOTEBaggingClassifier
from skmultiflow.evaluation import EvaluatePrequential
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score,accuracy_score,roc_auc_score,confusion_matrix
%matplotlib notebook

# load training and test data

In [ ]:
file_paths = '..//preprocessed_data/Sebastian/Preprocessed_without_anchor/binary/'
filenames = glob.glob(file_paths+"*[0-9].csv",recursive = True)
list_of_dfs = [pd.read_csv(filename,sep=';',header=0) for filename in filenames]
data_all = pd.DataFrame()
for dataframe, filename in zip(list_of_dfs, filenames):
    data_all=data_all.append(dataframe.iloc[:,-5:])
data=pd.DataFrame(data_all)
data.reset_index(drop=True,inplace=True)
len(data)

In [ ]:
file_paths = '..//data/processed/'
filenames = glob.glob(file_paths+"*[0-9].csv",recursive = True)
list_of_dfs = [pd.read_csv(filename,sep=';',header=0) for filename in filenames]
data_t_all = pd.DataFrame()
for dataframe, filename in zip(list_of_dfs, filenames):
    data_t_all=data_t_all.append(dataframe.iloc[:,-5:])
data_t=pd.DataFrame(data_t_all)
data_t.reset_index(drop=True,inplace=True)
len(data_t)

# Singular Spectrum Analysis (SSA)

In [ ]:
X = data.iloc[:,:4].transpose()
X_test = data_t.iloc[:,:4].transpose()
groups = [np.arange(i, i + 4) for i in range(0, 11, 4)]
ssa = SingularSpectrumAnalysis(window_size=15, groups=groups)
ssa.fit(X)
X_ssa = ssa.transform(X)
X_ssa_test=ssa.transform(X_test)
X_ssa.shape

In [ ]:
plt.figure(figsize=(10, 6))

ax1 = plt.subplot(121)
ax1.plot(X.iloc[0,:], 'o-', label='Original')
ax1.legend(loc='best', fontsize=14)

ax2 = plt.subplot(122)
for i in range(len(groups)):
    ax2.plot(X_ssa[0, i], 'o--', label='SSA {0}'.format(i + 1))
ax2.legend(loc='best', fontsize=14)

plt.suptitle('Singular Spectrum Analysis', fontsize=20)

plt.tight_layout()
plt.subplots_adjust(top=0.88)
plt.show()

In [ ]:
final_df=X_ssa[0, 0]